# Day 1

## Part 1

You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about ghosts a few minutes ago.

One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of network of labeled nodes.

It seems like you're meant to use the left/right instructions to navigate the network. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: AAA and ZZZ. You feel like AAA is where you are now, and you have to follow the left/right instructions until you reach ZZZ.

This format defines each node of the network individually. For example:

```
RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
```
Starting with AAA, you need to look up the next element based on the next left/right instruction in your input. In this example, start with AAA and go right (R) by choosing the right element of AAA, CCC. Then, L means to choose the left element of CCC, ZZZ. By following the left/right instructions, you reach ZZZ in 2 steps.

Of course, you might not find ZZZ right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: RL really means RLRLRLRLRLRLRLRL... and so on. For example, here is a situation that takes 6 steps to reach ZZZ:

```
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
```
Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?

In [1]:
def traverse_branches(branches: dict, starting_node: str, directions: str, moves: int = 0) -> int:
    current_node = starting_node

    for direction in directions:
        moves += 1
        if direction == "L":
            next_node = branches[current_node][0]
        elif direction == "R":
            next_node = branches[current_node][1]

        # print(f"{moves=}: {current_node=} -> {direction} -> {next_node=}")
        current_node = next_node
        if current_node == "ZZZ":
            # print("hooray!")
            break

    if current_node != "ZZZ":
        # time to do it all over again
        return traverse_branches(
            branches=branches,
            starting_node=current_node,
            directions=directions,
            moves=moves,
        )

    return moves


def create_branches(branch_data: list[str]) -> dict:
    branches = {}
    for branch in branch_data:
        node_name, left_right = branch.split(" = ")
        left, right = left_right.strip("()").split(", ")
        branches[node_name] = (left, right)
    return branches


def load_inputs(inputs: str) -> tuple[str, list[str]]:
    directions, branch_data = inputs.split("\n\n")
    branch_lines = branch_data.split("\n")
    return directions, branch_lines


def solve(inputs: str) -> int:
    directions, branch_lines = load_inputs(inputs)
    branches = create_branches(branch_lines)
    moves = traverse_branches(branches, "AAA", directions)
    return moves


test_inputs = """RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)"""
test_solution = solve(test_inputs)
assert test_solution == 2

In [2]:
test_inputs2 = """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""
test_solution2 = solve(test_inputs2)
assert test_solution2 == 6

In [3]:
inputs = open("../inputs/8.txt").read()
solution = solve(inputs)
solution

14681

## Part 2

The sandstorm is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take significantly more steps to escape!

What if the map isn't for people - what if the map is for ghosts? Are ghosts even bound by the laws of spacetime? Only one way to find out.

After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in A is equal to the number ending in Z! If you were a ghost, you'd probably just start at every node that ends with A and follow all of the paths at the same time until they all simultaneously end up at nodes that end with Z.

For example:

```
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
```
Here, there are two starting nodes, 11A and 22A (because they both end with A). As you follow each left/right instruction, use that instruction to simultaneously navigate away from both nodes you're currently on. Repeat this process until all of the nodes you're currently on end with Z. (If only some of the nodes you're on end with Z, they act like any other node and you continue as normal.) In this example, you would proceed as follows:

- Step 0: You are at 11A and 22A.
- Step 1: You choose all of the left paths, leading you to 11B and 22B.
- Step 2: You choose all of the right paths, leading you to 11Z and 22C.
- Step 3: You choose all of the left paths, leading you to 11B and 22Z.
- Step 4: You choose all of the right paths, leading you to 11Z and 22B.
- Step 5: You choose all of the left paths, leading you to 11B and 22C.
- Step 6: You choose all of the right paths, leading you to 11Z and 22Z.
So, in this example, you end up entirely on nodes that end in Z after 6 steps.

Simultaneously start on every node that ends with A. How many steps does it take before you're only on nodes that end with Z?

In [4]:
# I thought having a class would make this easier but part 2 had other ideas


class NodeBranch:
    def __init__(self, branches: dict, directions: str, moves: int = 0):
        self.branches = branches
        self.directions = directions
        self.moves = moves

    def traverse(self, starting_nodes: list[str]) -> int:
        current_nodes = starting_nodes

        for direction in self.directions:
            for i, current_node in enumerate(starting_nodes):
                if direction == "L":
                    next_node = self.branches[current_node][0]
                elif direction == "R":
                    next_node = self.branches[current_node][1]

                # print(f"branch{i} @ move{self.moves}: {current_node=} -> {direction} -> {next_node=}")
                current_node = next_node
                current_nodes[i] = current_node
                if all([n.endswith("Z") for n in current_nodes]):
                    print(f"done!")
                    break
                self.moves += 1

        if not all([n.endswith("Z") for n in current_nodes]):
            # time to do it all over again
            return self.traverse(current_nodes)

        return self.moves


test_inputs2a = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)"""

test_directions2a, test_branch_lines2a = load_inputs(test_inputs2a)
test_branches2a = create_branches(test_branch_lines2a)

test_starting_nodes2a = [n for n in test_branches2a.keys() if n.endswith("A")]
print(test_starting_nodes2a)

test_nb = NodeBranch(test_branches2a, test_directions2a)
test_moves2a = test_nb.traverse(test_starting_nodes2a)

assert test_moves2a == 6

['11A', '22A']
done!


In [5]:
# import sys
# sys.setrecursionlimit(10_000_000)

directions2, branch_lines2 = load_inputs(inputs)
branches2 = create_branches(branch_lines2)

starting_nodes2 = [n for n in branches2.keys() if n.endswith("A")]
moves = []
for node in starting_nodes2:
    node_moves = traverse_branches(branches2, node, directions2)
    print(f"{node=} {node_moves=}")
    moves.append(node_moves)
node_moves